In [38]:
import numpy as np
import pandas as pd
from scipy import signal, ndimage, interpolate, stats, spatial
import sys, os, time, json
from pathlib import Path
import pickle as pkl

sys.path.append('../PreProcessing/')
sys.path.append('../Lib/')
sys.path.append('../Analyses/')

import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt

import TreeMazeFunctions as TMF
import spike_functions as SF
import analyses_table as AT
import plot_functions as PF
from importlib import reload  # Python 3.4+ only.


In [40]:

sns.set(style="whitegrid",font_scale=1,rc={
    'axes.spines.bottom': False,
'axes.spines.left': False,
'axes.spines.right': False,
'axes.spines.top': False,
'axes.edgecolor':'0.5'})

In [3]:
oakPaths = AT.getOakPaths()

In [97]:
def fillRow(df,cnt,an,task,date,uType,se,tt,cl,seID,depth):
    df.loc[cnt,'uniqueID'] = cnt
    df.loc[cnt,'an'] = an
    df.loc[cnt,'date'] = date
    df.loc[cnt,'task'] = task
    df.loc[cnt,'uType'] = uType
    df.loc[cnt,'se'] = se
    df.loc[cnt,'tt']=tt
    df.loc[cnt,'cl']=c
    df.loc[cnt,'seUnitID'] = seID
    df.loc[cnt,'Depth'] = depth

    return df


Create Tables aligning tetrode distances to session dates.

In [100]:

animals = ['Li','Cl','Ne']
tasks = ['OF','T3h','T3g','T3j','T3i']


cnt = 0
unitIDs = pd.DataFrame(columns=['an','task','date','se','tt','cl','seUnitID','uType','uniqueID','Depth','manualCheck'])
for an in animals:
    fn = oakPaths['Root'] / (an+'_SessionPaths.pkl')
    with fn.open(mode="rb") as f:
        sePaths = pkl.load(f)

    CT_Path =  oakPaths['Clustered'] / an / (an+'_ClusteringSummary.json')
    with CT_Path.open() as f:
        CT = json.load(f)

    # load raw tetrode data
    ttPath = (oakPaths['Clustered'] / (an + '_Tetrodes.csv'))
    ttInfo =  pd.read_csv(ttPath)
    dates = pd.Series(list(CT[an].keys()))
    try:
        fulldates = pd.to_datetime(dates,infer_datetime_format = True, unit='D')
        ttInfo['FullDate'] = pd.to_datetime(ttInfo['Date'],infer_datetime_format=True, unit='D')
    except:
        fulldates = pd.to_datetime(dates,infer_datetime_format = True)
        ttInfo['FullDate'] = pd.to_datetime(ttInfo['Date'],infer_datetime_format=True)
        
    ttInfo = ttInfo.sort_values('FullDate')
    x = []
    for d in ttInfo['FullDate']:
        x.append(d.strftime('%m%d%y'))
    ttInfo['Date'] = x
    ttInfo = ttInfo.set_index('Date')

    ttCols = ['depth_tt' + str(x) for x in np.arange(1,17) ]
    
    # organize tetrode data to dates
    seTTInfo = pd.DataFrame(columns=['Dates','FullDates'] + tasks +  ttCols)
    seTTInfo['FullDates'] = fulldates.values
    seTTInfo['Dates'] = dates.to_frame()
    seTTInfo = seTTInfo.sort_values('FullDates')
    
    for t in tasks:
        seTTInfo[t] = 0
        
    seTTInfo = seTTInfo.set_index('Dates')
    for date in seTTInfo.index:
        for task in CT[an][date].keys():
            seTTInfo.loc[date,task] = CT[an][date][task]['nCells'] + CT[an][date][task]['nMua'] 
        for tt in np.arange(1,17):
            depth = ttInfo.loc[date,('TT{}_overall').format(tt)]
            seTTInfo.loc[date,'depth_tt'+str(tt)] = depth
    
    seTTInfo.to_csv(str(oakPaths['Clustered'] / (an + '_SeTTInfo.csv' )) )

    for date in seTTInfo.index:
        for task,seInfo in CT[an][date].items():
            se = '{}_{}_{}'.format(an,task,date)
            seID = 0
            for tt,cl in seInfo['cell_IDs'].items():
                depth = ttInfo.loc[date,('TT{}_overall').format(tt)]
                if cl:
                    for c in cl:
                        unitIDs = fillRow(unitIDs,cnt,an,task,date,'Cell',se,tt,c,seID,depth)
                        seID +=1
                        cnt+=1
            for tt,cl in seInfo['mua_IDs'].items():
                depth = ttInfo.loc[date,('TT{}_overall').format(tt)]
                if cl:
                    for c in cl:
                        unitIDs = fillRow(unitIDs,cnt,an,task,date,'Mua',se,tt,c,seID,depth)
                        seID +=1
                    cnt+=1
unitIDs.to_csv(str(oakPaths['Clustered'] / 'UnitTable.csv' ))


In [101]:
unitIDs

,an,task,date,se,tt,cl,seUnitID,uType,uniqueID,Depth,manualCheck
0,Li,OF,052818,Li_OF_052818,7,5,0,Cell,0,17.25,NaN
1,Li,OF,052818,Li_OF_052818,8,2,1,Cell,1,16.25,NaN
2,Li,OF,052818,Li_OF_052818,8,4,2,Cell,2,16.25,NaN
3,Li,OF,052818,Li_OF_052818,14,0,3,Cell,3,16.75,NaN
4,Li,OF,052818,Li_OF_052818,14,1,4,Mua,4,16.75,NaN
5,Li,T3g,052818,Li_T3g_052818,7,0,0,Cell,5,17.25,NaN
6,Li,T3g,052818,Li_T3g_052818,7,8,1,Cell,6,17.25,NaN
7,Li,T3g,052818,Li_T3g_052818,8,0,2,Cell,7,16.25,NaN
8,Li,T3g,052818,Li_T3g_052818,14,11,3,Cell,8,16.75,NaN
9,Li,T3g,052818,Li_T3g_052818,2,9,4,Mua,9,16.5,NaN


compute session spikes.

In [4]:
animals = ['Li','Cl','Ne','Al','Ca']
for an in animals:
    CT_Path =  oakPaths['Clustered'] / an / (an+'_ClusteringSummary.json')
    with CT_Path.open() as f:
        CT = json.load(f)
        
    fn = oakPaths['Root'] / (an+'_SessionPaths.pkl')
    with fn.open(mode="rb") as f:
        sePaths = pkl.load(f)
    
    for se,paths in sePaths.items():
        try:
            an,task,date=se.split('_')
            _ = SF.getSessionSpikes(paths)
        except:
            pass

Spikes Files not Found or overwrite=1, creating them.
Spikes Files not Found or overwrite=1, creating them.
Spikes Files not Found or overwrite=1, creating them.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Spikes Files not Found or overwrite=1, creating them.
Spikes Files not Found or overwrite=1, creating them.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Loaded Spike Files.
Spikes Files not Found or overwrite=1, creating them.
Loaded Spike Fil

plot the spikes wf and isi

In [86]:
#animals = ['Li','Cl','Ne','Al','Ca']
animals = ['Cl','Al','Ca']
for an in animals:
    CT_Path =  oakPaths['Clustered'] / an / (an+'_ClusteringSummary.json')
    with CT_Path.open() as f:
        CT = json.load(f)
    seTTInfo = pd.read_csv(str(oakPaths['Clustered'] / (an + '_SeTTInfo.csv' )),dtype={'Dates':str})
    seTTInfo=seTTInfo.set_index('Dates')
        
    fn = oakPaths['Root'] / (an+'_SessionPaths.pkl')
    with fn.open(mode="rb") as f:
        sePaths = pkl.load(f)
    
    for date in seTTInfo.index:
        for task in CT[an][date].keys():
            se = '{}_{}_{}'.format(an,task,date)
            seDat = AT.loadSessionData(sePaths[se],vars = ['wfi'])['wfi']

            for ut in ['Cell','Mua']:
                nUnits = len(seDat[ut])
                f,ax = plt.subplots(nUnits,2,figsize=(12,5*nUnits))
                ax = ax.flatten()
                for unit,dat in seDat[ut].items():
                    tt = dat['tt']
                    cl = dat['cl']
                    PF.plotSpikeWFs(dat,ax=ax[2*unit])
                    PF.plotISIh(dat,ax=ax[2*unit+1])
                    ax[2*unit].set_title('{}: tt:{}, cl:{}, uT: {}, d={}'.format(se,tt,cl,ut,seTTInfo.loc[date,'depth_tt'+tt]  ))
                f.savefig(sePaths[se]['Plots'] / '{}_SpikeWF__{}.jpeg'.format(se,ut), bbox_inches='tight',pad_inches=0.2)
                plt.close(f)
        

FileNotFoundError: [Errno 2] File b'/mnt/o/giocomo/alexg/Clustered/Al_SeTTInfo.csv' does not exist: b'/mnt/o/giocomo/alexg/Clustered/Al_SeTTInfo.csv'